In [1]:
from matplotlib.patches import Circle, RegularPolygon
from matplotlib.projections import register_projection
from matplotlib.projections.polar import PolarAxes
from matplotlib.transforms import Affine2D
from matplotlib.ticker import FixedLocator
from matplotlib.spines import Spine
from matplotlib.path import Path
from math import pi

import matplotlib.pyplot as plt
import numpy as np
import PySimpleGUI as sg

In [2]:
def radar_factory(num_vars, frame='circle'):
    
    theta = np.linspace(0, 2*np.pi, num_vars, endpoint=False)
    
    class RadarAxes(PolarAxes):
        name = 'radar'

        def __init__(self, *args, **kwargs):
            super().__init__(*args, **kwargs)
            self.set_theta_zero_location('N')

        def fill(self, *args, closed=True, **kwargs):
            return super().fill(closed=closed, *args, **kwargs)

        def plot(self, *args, **kwargs):
            lines = super().plot(*args, **kwargs)
            for line in lines:
                self._close_line(line)

        def _close_line(self, line):
            x, y = line.get_data()
            if x[0] != x[-1]:
                x = np.concatenate((x, [x[0]]))
                y = np.concatenate((y, [y[0]]))
                line.set_data(x, y)

        def set_varlabels(self, labels):
            self.set_thetagrids(np.degrees(theta), labels)

        def _gen_axes_patch(self):
            if frame == 'circle':
                return Circle((0.5, 0.5), 0.5)
            elif frame == 'polygon':
                return Circle((0.5, 0.5), 0.5)
            else:
                raise ValueError("unknown value for 'frame': %s" % frame)

        def draw(self, renderer):
            if frame == 'polygon':
                gridlines = self.yaxis.get_gridlines()
                for gl in gridlines:
                    gl.get_path()._interpolation_steps = num_vars
            super().draw(renderer)

        def _gen_axes_spines(self):
            if frame == 'circle':
                return super()._gen_axes_spines()
            elif frame == 'polygon':
                spine = Spine(axes=self,
                              spine_type='circle',
                              path=Path.unit_regular_polygon(num_vars))
                spine.set_transform(Affine2D().rotate(pi/8).scale(0.5).translate(.5, .5)
                                    + self.transAxes)

                return {'polar': spine}
            else:
                raise ValueError("unknown value for 'frame': %s" % frame)

    register_projection(RadarAxes)
    return theta

In [3]:
def web1(file_name, col_names, numbers):
    dict1 = {col: 0 for col in col_names}
    N = 8
    theta = radar_factory(8, frame='polygon')

    angles = np.linspace(0, 2 * pi, N, endpoint=False)
    angles_mids = angles + (angles[1] / 2)

    fig, ax = plt.subplots(figsize=(11, 8), subplot_kw=dict(projection='radar'))
    ax.xaxis.set_minor_locator(FixedLocator(angles))
    fig.subplots_adjust(top=0.95, bottom=0.05, left=0.07, right=0.85)
    plt.ylim([0, 10])

    ax.set_theta_offset(pi/8-pi/4)
    ax.set_theta_direction(1)
    ax.set_xticks(angles_mids)
    ax.set_xticklabels(dict1.keys())
    ax.grid(True, axis='x', which='minor')
    ax.grid(False, axis='x', which='major')
    ax.grid(True, axis='y', which='major')
    ax.set_rgrids([2,4,6,8,10], [])
    #ax.set_title(file_name,  position=(0.5, 1.1), ha='center')
    color1='springgreen'
    color2='seagreen'
    color3='tomato'
    color4='red'
    for i in range(6):
        ax.fill_between(angles[i:i+2], 0, numbers[i] , facecolor=color1, edgecolor=color2, linewidth=2)
    ax.fill_between(angles[5:7], 0, numbers[5], facecolor=color3, edgecolor=color4, linewidth=2)
    ax.fill_between(angles[6:8], 0, numbers[6], facecolor=color3, edgecolor=color4, linewidth=2)
    ax.fill_between([angles[7],(angles[7]+angles[1])], 0, numbers[7], facecolor=color3, edgecolor=color4, linewidth=2)

    for index, label in enumerate(ax.get_xticklabels()):
        label.set_weight("bold")
        label.set_fontsize(14)
        if index in [0, 1, 7]:
            label.set_horizontalalignment("left")
        elif index in [3, 4, 5]:
            label.set_horizontalalignment("right")

    plt.savefig(file_name, transparent=True)

In [4]:
def window_string(degrees, text, text_size, input_size):
    return ([sg.Text(degrees), sg.InputText(text, key='col'+degrees, size=(text_size,1)),
             sg.InputText(0, key=degrees, size=(input_size,1), enable_events=True)])

def make_window():
    layout = [
        [sg.HorizontalSeparator()],
        window_string("1", "Дисконтирование / устаревших / решений", 40, 3),
        window_string("2", "Давление на / независимого / партнера", 40, 3),
        window_string("3", "Конструирование / компаний", 40, 3),
        window_string("4", "Препарирование / технологий", 40, 3),
        window_string("5", "Скрининг / шансов", 40, 3),
        window_string("6", "Борьба за / привилегии", 40, 3),
        window_string("7", "Фиктивная / деятельность", 40, 3),
        window_string("8", "Отношения / вместо дела", 40, 3),
        [sg.Text("Название"), sg.InputText("PizzaChart", key='name', size=(30, 1)), sg.Text(".png")],
        [sg.HorizontalSeparator()],
        [sg.Output(size=(46, 8))],
        [sg.Submit("Пицца")]
        ]
    return sg.Window('PizzaChart v1.0', layout)

def check_input(value):
    try:
        value = float(value)
        if value > 10:
            window[event].update(10)
    except:
        if value == '':
            pass
        else:
            window[event].update(0)

In [5]:
sg.theme('Reddit')
window = make_window()
while True:
    event, values = window.read()
    if event in (None, 'Exit'):
        break
    if event in [str(i) for i in range(1,9)]:
        check_input(values[event])
    if event == 'Пицца':
        col_names = [values['col'+str(i)].replace(' / ', '\n') for i in range(1,9)]
        numbers = [float(values[str(i)]) if values[str(i)] != '' else 0 for i in range(1,9)]
        numbers = [int(x) if int(x) == x else x for x in numbers]
        name = values['name']+'.png'
        name = name.replace('\n', '')
        web1(name, col_names, numbers)
        print('Файл создан: ' + name)
        print('Значения =', numbers)
window.close()